# 1.xml格式数据转换为yolo格式

In [1]:
import os
import xml.etree.ElementTree as ET


def xml_to_txt(xml_file, class_mapping):
    with open(xml_file, encoding='utf-8') as f:
        # 加载xml文件
        tree = ET.parse(f)
        
    # 获取根元素
    root = tree.getroot()

    yolo_labels = []
    
    # 图片的实际宽和高
    w, h = 8192, 4096

    for obj in root.iter('object'):
        # 类别信息
        class_info = obj.find('name').text
        if class_info in class_mapping:
            c = class_mapping[class_info]
            bnd_box = obj.find('bndbox')
            
            # 坐标信息
            x_min = int(bnd_box.find('xmin').text)
            x_max = int(bnd_box.find('xmax').text)
            y_min = int(bnd_box.find('ymin').text)
            y_max = int(bnd_box.find('ymax').text)

            x_center = (x_min+x_max)/2/w
            y_center = (y_min+y_max)/2/h

            width = (x_max-x_min) / w
            height = (y_max-y_min) / h

            yolo_labels.append(f"{c} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}")
    return yolo_labels


class_mapping = {
    'constructionCrack': 0,
    "crack": 1
}
# 待转换文件
f = r'../example_images/Cam10-2023_05_02_12_12_03_852-90-588.xml'
labels = xml_to_txt(f, class_mapping)
new_file = os.path.join(os.path.dirname(f), os.path.basename(f)[:-3]+'txt')
with open(new_file, 'w') as f:
    for label in labels:
        f.write(label+'\n')